In [1]:
# Restart dfs
!stop-dfs.sh
!start-dfs.sh

Stopping namenodes on [0.0.0.0]
Stopping datanodes
Stopping secondary namenodes [dataeng-virtual-machine]
Starting namenodes on [0.0.0.0]
Starting datanodes
Starting secondary namenodes [dataeng-virtual-machine]


In [2]:
# Check java services running
!jps


72433 SparkSubmit
126293 NameNode
115524 SparkSubmit
28055 SparkSubmit
126614 SecondaryNameNode
126838 Jps
126394 DataNode


In [3]:
# Check Safe mode: should be OFF
!hdfs dfsadmin -safemode get


Safe mode is OFF


In [4]:
# If Safe mode is ON --> execute hdfs dfsadmin -safemode leave
!hdfs dfsadmin -safemode leave


Safe mode is OFF


In [5]:
# Check hdfs home files
!hdfs dfs -ls


In [6]:
# Create hdfs directories
!hdfs dfs -mkdir -p \
    kaggle/datasets \
    kaggle/splits

In [7]:
!hdfs dfs -ls kaggle/


Found 2 items
drwxr-xr-x   - dataeng supergroup          0 2025-01-19 20:55 kaggle/datasets
drwxr-xr-x   - dataeng supergroup          0 2025-01-19 20:55 kaggle/splits


In [8]:
# Download kaggle dataset into hdfs

!kaggle datasets download -d robertvici/indonesia-top-ecommerce-unicorn-tweets -p ~/kaggle-datasets
!unzip -o ~/kaggle-datasets/indonesia-top-ecommerce-unicorn-tweets.zip -d ~/kaggle-datasets
!hdfs dfs -put ~/kaggle-datasets/*.json kaggle/datasets/

Dataset URL: https://www.kaggle.com/datasets/robertvici/indonesia-top-ecommerce-unicorn-tweets
License(s): copyright-authors
indonesia-top-ecommerce-unicorn-tweets.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /home/dataeng/kaggle-datasets/indonesia-top-ecommerce-unicorn-tweets.zip
  inflating: /home/dataeng/kaggle-datasets/ShopeeID.json  
  inflating: /home/dataeng/kaggle-datasets/bliblidotcom.json  
  inflating: /home/dataeng/kaggle-datasets/bukalapak.json  
  inflating: /home/dataeng/kaggle-datasets/lazadaID.json  
  inflating: /home/dataeng/kaggle-datasets/tokopedia.json  


In [9]:
# Show list of dataset files downloaded
!hdfs dfs -ls /user/dataeng/kaggle/datasets


Found 5 items
-rw-r--r--   3 dataeng supergroup  233134634 2025-01-19 20:55 /user/dataeng/kaggle/datasets/ShopeeID.json
-rw-r--r--   3 dataeng supergroup   56417505 2025-01-19 20:55 /user/dataeng/kaggle/datasets/bliblidotcom.json
-rw-r--r--   3 dataeng supergroup   46806779 2025-01-19 20:55 /user/dataeng/kaggle/datasets/bukalapak.json
-rw-r--r--   3 dataeng supergroup  118395100 2025-01-19 20:55 /user/dataeng/kaggle/datasets/lazadaID.json
-rw-r--r--   3 dataeng supergroup   29228846 2025-01-19 20:55 /user/dataeng/kaggle/datasets/tokopedia.json


In [10]:
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, lower, regexp_replace, count
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import lit
import tensorflow as tf
import numpy as np
from keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle


2025-01-19 20:55:49.856440: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-19 20:55:49.858435: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-19 20:55:49.872684: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-19 20:55:49.914038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737294949.984325  125677 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737294950.00

In [11]:
# !export SPARK_LOCAL_IP="192.168.241.136"

In [12]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("E-Commerce Engagement Prediction ML") \
    .getOrCreate()

25/01/19 20:56:05 WARN Utils: Your hostname, dataeng-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.241.136 instead (on interface ens33)
25/01/19 20:56:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/19 20:56:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
# spark.sparkContext.setLogLevel("INFO")

In [14]:
# Load datasets from HDFS

blibli_df = spark.read.json('hdfs://localhost:9000/user/dataeng/kaggle/datasets/bliblidotcom.json')
bukalapak_df = spark.read.json('hdfs://localhost:9000/user/dataeng/kaggle/datasets/bukalapak.json')
lazadaID_df = spark.read.json('hdfs://localhost:9000/user/dataeng/kaggle/datasets/lazadaID.json')
shopeeID_df = spark.read.json('hdfs://localhost:9000/user/dataeng/kaggle/datasets/ShopeeID.json')
tokopedia_df = spark.read.json('hdfs://localhost:9000/user/dataeng/kaggle/datasets/tokopedia.json')

25/01/19 20:56:20 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [15]:
# Add a new column to identify the company source
blibli_df = blibli_df.withColumn('source', lit('blibli'))
bukalapak_df = bukalapak_df.withColumn('source', lit('bukalapak'))
lazadaID_df = lazadaID_df.withColumn('source', lit('lazadaID'))
shopeeID_df = shopeeID_df.withColumn('source', lit('shopeeID'))
tokopedia_df = tokopedia_df.withColumn('source', lit('tokopedia'))

# Merge datasets using union (axis=0 equivalent in Spark)
merged_df = blibli_df.union(bukalapak_df).union(lazadaID_df).union(shopeeID_df).union(tokopedia_df)

# Show merged data
merged_df.show()

25/01/19 20:56:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+-------------------+-------------+----------+---+--------------------+-------------------+-----------+--------------------+-----------------+----------+----+--------------------+-----+---------+-------------+--------------------+-------+------------+----------+--------------+------+--------+--------+----------+---------+---------+--------------------+--------------------+---------+-------+----------+------------+-----+
|cashtags|    conversation_id|   created_at|      date|geo|            hashtags|                 id|likes_count|                link|         mentions|      name|near|              photos|place|quote_url|replies_count|            reply_to|retweet|retweet_date|retweet_id|retweets_count|source|    time|timezone|trans_dest|trans_src|translate|               tweet|                urls|  user_id|user_rt|user_rt_id|    username|video|
+--------+-------------------+-------------+----------+---+--------------------+-------------------+-----------+--------------------+-

In [16]:
# DataFrame structure
print("Schema of the merged DataFrame:")
merged_df.printSchema()

Schema of the merged DataFrame:
root
 |-- cashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: long (nullable = true)
 |-- date: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: long (nullable = true)
 |-- likes_count: long (nullable = true)
 |-- link: string (nullable = true)
 |-- mentions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- near: string (nullable = true)
 |-- photos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- place: string (nullable = true)
 |-- quote_url: string (nullable = true)
 |-- replies_count: long (nullable = true)
 |-- reply_to: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- user_id: string (nullable = true)
 |    |    |-- userna

In [17]:
# Number of rows
row_count = merged_df.count()
print(f"Number of rows in the merged DataFrame: {row_count}")

Number of rows in the merged DataFrame: 541180


In [18]:
# Clean tweet text
def clean_text(text):
    return text.lower().replace("#", "").strip()

clean_text_udf = udf(clean_text, StringType())

# Apply text cleaning and create new features
data_cleaned = merged_df.withColumn("clean_tweet", clean_text_udf(col("tweet"))) \
                       .withColumn("engagement", col("replies_count") + col("retweets_count") + col("likes_count"))

# Select relevant features
selected_data = data_cleaned.select(
    col("clean_tweet").alias("text"),
    col("replies_count").alias("replies"),
    col("retweets_count").alias("retweets"),
    col("likes_count").alias("likes"),
    col("engagement").alias("target"),
    col("hashtags"),    
    col("source")
)

# Show processed data
selected_data.show()


+--------------------+-------+--------+-----+------+--------------------+------+
|                text|replies|retweets|likes|target|            hashtags|source|
+--------------------+-------+--------+-----+------+--------------------+------+
|sudah kepikiran m...|      0|       0|    1|     1| [#ramadanlebihbaik]|blibli|
|belanja pake kart...|      0|       0|    0|     0| [#ramadanlebihbaik]|blibli|
|udah seminggu ram...|      0|       0|    1|     1|[#ramadan, #ramad...|blibli|
|catet tanggal mai...|      0|       1|    1|     2|                  []|blibli|
|yuk! gabung di ng...|      1|       1|    1|     3|    [#ngobrolbareng]|blibli|
|udah bosen banget...|      1|       2|    1|     4|                  []|blibli|
|long weekend! bak...|      1|       1|    3|     5|       [#dirumahaja]|blibli|
|selamat hari buru...|      0|       0|    3|     3|       [#mayday2020]|blibli|
|cek di sini, ya h...|      0|       0|    0|     0|                  []|blibli|
|buang semua racun...|      

In [19]:
# Split dataset
train_data, validate_data, test_data = selected_data.randomSplit([0.7, 0.15, 0.15], seed=42)

# Save splits for later use
train_data.write.json("kaggle/splits/train.json", mode="overwrite")
validate_data.write.json("kaggle/splits/validate.json", mode="overwrite")
test_data.write.json("kaggle/splits/test.json", mode="overwrite")

In [20]:
# check null values

merged_df.filter(
    (merged_df.likes_count.isNull()) | 
    (merged_df.replies_count.isNull()) | 
    (merged_df.retweets_count.isNull())
).show()


+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+-------------+--------+-------+------------+----------+--------------+------+----+--------+----------+---------+---------+-----+----+-------+-------+----------+--------+-----+
|cashtags|conversation_id|created_at|date|geo|hashtags| id|likes_count|link|mentions|name|near|photos|place|quote_url|replies_count|reply_to|retweet|retweet_date|retweet_id|retweets_count|source|time|timezone|trans_dest|trans_src|translate|tweet|urls|user_id|user_rt|user_rt_id|username|video|
+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+-------------+--------+-------+------------+----------+--------------+------+----+--------+----------+---------+---------+-----+----+-------+-------+----------+--------+-----+
+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+

In [21]:
# Change null value with 0 (if any)
merged_df = merged_df.fillna({"likes_count": 0, "replies_count": 0, "retweets_count": 0})


In [22]:
from pyspark.sql import functions as F

# Check negative value
merged_df.filter((F.col("likes_count") < 0) | (F.col("replies_count") < 0) | (F.col("retweets_count") < 0)).show()


+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+-------------+--------+-------+------------+----------+--------------+------+----+--------+----------+---------+---------+-----+----+-------+-------+----------+--------+-----+
|cashtags|conversation_id|created_at|date|geo|hashtags| id|likes_count|link|mentions|name|near|photos|place|quote_url|replies_count|reply_to|retweet|retweet_date|retweet_id|retweets_count|source|time|timezone|trans_dest|trans_src|translate|tweet|urls|user_id|user_rt|user_rt_id|username|video|
+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+-------------+--------+-------+------------+----------+--------------+------+----+--------+----------+---------+---------+-----+----+-------+-------+----------+--------+-----+
+--------+---------------+----------+----+---+--------+---+-----------+----+--------+----+----+------+-----+---------+

In [23]:
# Change negative value with 0 (if any)
for col in ["likes_count", "replies_count", "retweets_count"]:
    merged_df = merged_df.withColumn(col, F.when(F.col(col) < 0, 0).otherwise(F.col(col)))
    

In [24]:
# Matching target engagement definitions in Spark DataFrame
blibli_df = blibli_df.withColumn("engagement", F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
bukalapak_df = bukalapak_df.withColumn("engagement", F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
lazadaID_df = lazadaID_df.withColumn("engagement",   F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
shopeeID_df = shopeeID_df.withColumn("engagement",   F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))
tokopedia_df = tokopedia_df.withColumn("engagement", F.col("likes_count") + F.col("replies_count") + F.col("retweets_count"))


In [25]:
# Load train data (convert Spark DataFrame to Pandas)
train_df = train_data.toPandas()

# Tokenize and vectorize text (fit on original text, not the padded sequences)
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_df["text"])  # Fit tokenizer on the raw text data

# Convert texts to sequences
X_train = tokenizer.texts_to_sequences(train_df["text"])

# Pad the sequences to ensure uniform length
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post')

y_train = np.array(train_df["target"])


In [26]:
# Define a simple Neural Network model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1, activation="linear")
])

In [27]:
# Compile the model
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

2025-01-19 20:58:37.996048: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [28]:
vocab_size = 42500  # Customize with your tokenizer
embedding_dim = 128
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

In [29]:
# Example: Tokenize and pad the input text
max_vocab_size = 5000
tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<UNK>")
tokenizer.fit_on_texts(train_df["text"])  # 'train_df["text"]' should be a list of strings

# Example of saving the tokenizer
with open('../models/tokenizer_notebook.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

X_train_sequences = tokenizer.texts_to_sequences(train_df["text"])
X_train = pad_sequences(X_train_sequences, padding='post')

# Ensure y_train is in the correct format (e.g., a numpy array)
y_train = np.array(train_df["target"])  # Adjust this based on your target column

# Train the model !!!
model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 205s 17ms/step - loss: 965121.7500 - mae: 31.1688
Epoch 2/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 197s 17ms/step - loss: 691859.5625 - mae: 29.4899
Epoch 3/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 197s 17ms/step - loss: 717546.7500 - mae: 29.6016
Epoch 4/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 197s 17ms/step - loss: 503083.4375 - mae: 26.5202
Epoch 5/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 198s 17ms/step - loss: 1040871.4375 - mae: 32.5972
Epoch 6/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 198s 17ms/step - loss: 940018.0625 - mae: 31.2613
Epoch 7/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 201s 17ms/step - loss: 609515.5625 - mae: 28.4569
Epoch 8/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 198s 17ms/step - loss: 703785.1250 - mae: 31.8703
Epoch 9/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 195s 16ms/step - loss: 451232.2812 - mae: 28.5009
Epoch 10/10
11833/11833 ━━━━━━━━━━━━━━━━━━━━ 196s 17ms/step - loss: 522296.0938 - mae: 30.4058


In [30]:
# Save the model with a valid file extension in local server
model.save("../models/e-commerce-engagement_model_notebook.keras")  # For the native Keras format

In [31]:
# Export to save final model
model.export("../saved_model_notebook/1")

INFO:tensorflow:Assets written to: ../saved_model_notebook/1/assets


INFO:tensorflow:Assets written to: ../saved_model_notebook/1/assets


Saved artifact at '../saved_model_notebook/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 60), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  130002563162064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130002563162256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130002563163408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130002563162448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  130002563164752: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [32]:
# Stop the Spark session
spark.stop()